In [27]:
from LinkedBinaryTree import * 
import random
from game_tree import *


def build_set_tree_with_weights(tree):
    construct_tree(tree)

    visited = set()

    def assign_weights(node, win_on_receive_weight, momentum):
        if node is None or node in visited:
            return

        visited.add(node)

        if node.data == "WIN":
            node.right_weight = 1.0
            node.left_weight = 0.0
        elif node.data == "LOSE":
            node.right_weight = 0.0
            node.left_weight = 1.0
        else:
            node.right_weight = 0.5  # Default for winning
            node.left_weight = 0.5 # Default for losing

            if "40" in node.data and "0" not in node.data:
                node.right_weight += win_on_receive_weight

            if "15" in node.data or "30" in node.data:  # Example condition for momentum
                node.right_weight += momentum

            # Adjust weights if momentum is against the player
            if "0-" in node.data or "15-" in node.data or "30-0" in node.data:
                node.right_weight -= momentum
                node.left_weight += momentum

            # Normalize weights to ensure they sum to 1
            total_weight = node.right_weight + node.left_weight
            if total_weight > 0:
                node.right_weight /= total_weight
                node.left_weight /= total_weight

        assign_weights(node.left, win_on_receive_weight, momentum)
        assign_weights(node.right, win_on_receive_weight, momentum)

    assign_weights(tree.root, 0.1, 0.05)

def simulate_set(tree):
    player_games = 0
    opponent_games = 0

    while True:
        current_node = tree.root

        while current_node:
            if current_node.data == "WIN":
                player_games += 1
                break
            elif current_node.data == "LOSE":
                opponent_games += 1
                break

            random_value = random.random()
            if random_value < current_node.right_weight and current_node.right:
                current_node = current_node.right
            elif current_node.left:
                current_node = current_node.left

        if player_games >= 6 and player_games - opponent_games >= 2:
            return "Player wins the set", player_games, opponent_games
        if opponent_games >= 6 and opponent_games - player_games >= 2:
            return "Opponent wins the set", player_games, opponent_games

binary_tree = LinkedBinaryTree()
build_set_tree_with_weights(binary_tree)

result, player_games, opponent_games = simulate_set(binary_tree)
print(result)
print(f"Player Games: {player_games}, Opponent Games: {opponent_games}")


Player wins the set
Player Games: 7, Opponent Games: 5


In [28]:

def predict_set_winner(tree, point_sequence):
    player_games = 0
    opponent_games = 0

    for point in point_sequence:
        current_node = tree.root

        while current_node:
            if current_node.data == "WIN":
                player_games += 1
                break
            elif current_node.data == "LOSE":
                opponent_games += 1
                break

            if point == "WIN":
                if current_node.right:
                    current_node = current_node.right
                else:
                    break
            elif point == "LOSE":
                if current_node.left:
                    current_node = current_node.left
                else:
                    break

        if player_games >= 6 and player_games - opponent_games >= 2:
            return "Player wins the set", player_games, opponent_games
        if opponent_games >= 6 and opponent_games - player_games >= 2:
            return "Opponent wins the set", player_games, opponent_games

    return "Set in progress", player_games, opponent_games

binary_tree = LinkedBinaryTree()
build_set_tree_with_weights(binary_tree)

point_sequence = ["WIN", "LOSE", "WIN", "WIN", "LOSE", "LOSE", "WIN", "WIN", "LOSE", "WIN"]
result, player_games, opponent_games = predict_set_winner(binary_tree, point_sequence)
print(result)
print(f"Player Games: {player_games}, Opponent Games: {opponent_games}")


Player wins the set
Player Games: 6, Opponent Games: 4


In [29]:
def predict_set_winner_with_server(tree, match_data):
    player_games = 0
    opponent_games = 0
    current_game_score = {"player": 0, "opponent": 0}

    current_server = None
    current_game_no = None

    for _, row in match_data.iterrows():
        server = row["server"]
        game_no = row["game_no"]
        point_victor = row["point_victor"]

        # Check for new game
        if current_game_no is None or game_no != current_game_no:
            current_game_no = game_no
            current_game_score = {"player": 0, "opponent": 0}

        # Determine who wins the point
        if point_victor == 1:  # Player 1 wins the point
            current_game_score["player"] += 1
        elif point_victor == 2:  # Player 2 wins the point
            current_game_score["opponent"] += 1

        # Check if someone won the game
        if current_game_score["player"] >= 4 and current_game_score["player"] - current_game_score["opponent"] >= 2:
            player_games += 1
            current_game_score = {"player": 0, "opponent": 0}  # Reset game score
        elif current_game_score["opponent"] >= 4 and current_game_score["opponent"] - current_game_score["player"] >= 2:
            opponent_games += 1
            current_game_score = {"player": 0, "opponent": 0}  # Reset game score

        # Check if someone won the set
        if player_games >= 6 and player_games - opponent_games >= 2:
            return "Player wins the set", player_games, opponent_games
        if opponent_games >= 6 and opponent_games - player_games >= 2:
            return "Opponent wins the set", player_games, opponent_games

    return "Set in progress", player_games, opponent_games


In [30]:
def predict_set_winner_simulation(tree, match_data):
    player_games = match_data["p1_games"]
    opponent_games = match_data["p2_games"]
    player_score = match_data["p1_score"]
    opponent_score = match_data["p2_score"]

    while True:
        current_node = tree.root
        while current_node:
            random_value = random.random()
            if current_node.data == "WIN":
                player_score += 1
                if player_score == 4:  # Win game
                    player_games += 1
                    player_score = 0
                    opponent_score = 0
                break
            elif current_node.data == "LOSE":
                opponent_score += 1
                if opponent_score == 4:  # Lose game
                    opponent_games += 1
                    player_score = 0
                    opponent_score = 0
                break

            if random_value < current_node.right_weight and current_node.right:
                current_node = current_node.right
            elif current_node.left:
                current_node = current_node.left

        if player_games >= 6 and player_games - opponent_games >= 2:
            return "Player wins the set", player_games, opponent_games
        if opponent_games >= 6 and opponent_games - player_games >= 2:
            return "Opponent wins the set", player_games, opponent_games

binary_tree = LinkedBinaryTree()
build_set_tree_with_weights(binary_tree)


In [32]:
import pandas as pd
match_data = pd.read_csv("m1.csv")

# Predict winner by simulating forward
result, player_games, opponent_games = predict_set_winner_simulation(binary_tree, match_data)
print(result)
print(f"Player Games: {player_games}, Opponent Games: {opponent_games}")


TypeError: can only concatenate str (not "int") to str